# 图像卷积

## 互相关计算

In [3]:
import torch as tor
from torch import nn as tor_nn
from d2l import torch as d2l_tor

In [5]:
def corr2d(X, K):
    """计算二维互相关计算"""
    h, w = K.shape
    Y = tor.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)) # 以互相关来计算矩阵大小的初始化
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i : i + h, j : j + w] * K).sum() # X 是序列， K 是kenel是核
    return Y

验证一下

In [6]:
X = tor.tensor([[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]])
K = tor.tensor([[0.0,1.0],[2.0,3.0]])

corr2d(X, K) # 完美

tensor([[25., 31.],
        [43., 49.]])

## 卷积层实现

In [7]:
class Conv2D(tor_nn.Module):
    def __init__(self, kenel_size):
        super().__init__()
        self.weight = tor_nn.Parameter(tor.rand(kenel_size))
        self.bias = tor_nn.Parameter(tor.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

卷积层的简单应用(边缘寻找)，当然，真实操作中有Sobel算子，拉普拉斯算子，Priweet算子等诸多算子来进行边缘检测的计算，但我们在这里看向一些更为抽象本质的东西。请看下面的例子：

In [8]:
X = tor.ones((6,8))
X[:, 2:6] = 0

X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [9]:
K = tor.tensor([[1.0,-1.0]]) # 注意这里的二维是为了与输入一致

In [11]:
Y = corr2d(X, K)
Y # 我们看到他们检测到了边缘

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

当然，我们如果横着，这就找不到了

In [12]:
X.t()

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [13]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

如上，这是一个简单的数图过程，我们要进行机器学习我们还需要一些操作，我们尝试采用机器学习的方法将我们想要的kenel(和bias)学出来

In [14]:
conv2d = tor_nn.Conv2d(1, 1, kernel_size=(1,2),bias=False) #第一个是输入的Chanel，第二个是输出的Chanel

X = X.reshape((1,1,6,8)) # 加两个维度 通道维度和批量大小维度
Y = Y.reshape((1,1,6,7))

for i in range(10):
    y_hat = conv2d(X)
    l = (y_hat - Y)**2 # 差值的平方
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    #    直接访问数据         学习率     直接就是导
    if (i + 1) % 2 == 0:
        print(f'batch {i + 1}, loss {l.sum():.3f}')

batch 2, loss 10.528
batch 4, loss 3.208
batch 6, loss 1.129
batch 8, loss 0.431
batch 10, loss 0.171


看看学到的

In [15]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9466, -1.0312]])